In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
import sys

sys.path.insert(0, '../..')

## Load Dataset

In [4]:
from entity_embed.benchmarks import DBLPScholarStructuredBenchmark

benchmark = DBLPScholarStructuredBenchmark(data_dir_path="../data/")
benchmark

19:30:09 INFO:Extracting DBLP-Scholar-Structured...
19:30:09 INFO:Reading DBLP-Scholar-Structured record_dict...
19:30:10 INFO:Reading DBLP-Scholar-Structured train.csv...
19:30:10 INFO:Reading DBLP-Scholar-Structured valid.csv...
19:30:10 INFO:Reading DBLP-Scholar-Structured test.csv...


<DBLPScholarStructuredBenchmark> from http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/DBLP-GoogleScholar/dblp_scholar_exp_data.zip

## Preprocess

In [5]:
field_list = ['title', 'authors', 'venue', 'year']

In [6]:
import unidecode

def clean_str(s):
    return unidecode.unidecode(s).lower().strip()

for record_dict in [benchmark.train_record_dict, benchmark.valid_record_dict, benchmark.test_record_dict]:
    for record in record_dict.values():
        for field in field_list:
            record[field] = clean_str(record[field])

## Init Data Module

In [7]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [8]:
from entity_embed import PairNumericalizer

pair_numericalizer = PairNumericalizer(field_list)

In [9]:
batch_size = 32
eval_batch_size = 256
datamodule = benchmark.build_matcher_datamodule(
    pair_numericalizer=pair_numericalizer,
    batch_size=batch_size,
    eval_batch_size=eval_batch_size,
    random_seed=random_seed
)

## Training

In [10]:
from entity_embed import Matcher

model = Matcher(
    pair_numericalizer=pair_numericalizer
)

19:30:14 INFO:Load pretrained SentenceTransformer: stsb-distilbert-base
19:30:14 INFO:Did not find folder stsb-distilbert-base
19:30:14 INFO:Search model on server: http://sbert.net/models/stsb-distilbert-base.zip
19:30:14 INFO:Load SentenceTransformer from folder: /home/fjsj/.cache/torch/sentence_transformers/sbert.net_models_stsb-distilbert-base
19:30:15 INFO:Use pytorch device: cuda


In [11]:
trainer = model.fit(
    datamodule,
    min_epochs=5,
    max_epochs=100,
    check_val_every_n_epoch=1,
    early_stop_monitor="valid_f1_at_0.5",
    tb_save_dir='../tb_logs',
    tb_name=f'matcher-{benchmark.dataset_name}'
)

19:30:15 INFO:GPU available: True, used: True
19:30:15 INFO:TPU available: False, using: 0 TPU cores
19:30:15 INFO:Train positive pair count: 3207
19:30:15 INFO:Train negative pair count: 14016
19:30:15 INFO:Valid positive pair count: 1070
19:30:15 INFO:Valid positive pair count: 4672
19:30:15 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
19:30:18 INFO:
  | Name        | Type              | Params
--------------------------------------------------
0 | matcher_net | MatcherNet        | 66.4 M
1 | loss_fn     | BCEWithLogitsLoss | 0     
--------------------------------------------------
66.4 M    Trainable params
0         Non-trainable params
66.4 M    Total params
265.455   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

22:19:42 INFO:Loading the best validation model from ../tb_logs/matcher-DBLP-Scholar-Structured/version_0/checkpoints/epoch=41-step=22637.ckpt...
22:19:42 INFO:Load pretrained SentenceTransformer: stsb-distilbert-base
22:19:42 INFO:Did not find folder stsb-distilbert-base
22:19:42 INFO:Search model on server: http://sbert.net/models/stsb-distilbert-base.zip
22:19:42 INFO:Load SentenceTransformer from folder: /home/fjsj/.cache/torch/sentence_transformers/sbert.net_models_stsb-distilbert-base
22:19:43 INFO:Use pytorch device: cuda


In [12]:
model.validate(datamodule)

{'valid_f1_at_0.3': tensor(0.9510),
 'valid_f1_at_0.5': tensor(0.9527),
 'valid_f1_at_0.7': tensor(0.9526),
 'valid_f1_at_0.9': tensor(0.9537),
 'valid_precision_at_0.3': tensor(0.9488),
 'valid_precision_at_0.5': tensor(0.9541),
 'valid_precision_at_0.7': tensor(0.9566),
 'valid_precision_at_0.9': tensor(0.9637),
 'valid_recall_at_0.3': tensor(0.9533),
 'valid_recall_at_0.5': tensor(0.9514),
 'valid_recall_at_0.7': tensor(0.9486),
 'valid_recall_at_0.9': tensor(0.9439)}

## Testing

In [13]:
model.test(datamodule)

22:22:35 INFO:Test positive pair count: 1070
22:22:35 INFO:Test positive pair count: 4672


{'test_f1_at_0.3': tensor(0.9447),
 'test_f1_at_0.5': tensor(0.9440),
 'test_f1_at_0.7': tensor(0.9451),
 'test_f1_at_0.9': tensor(0.9434),
 'test_precision_at_0.3': tensor(0.9399),
 'test_precision_at_0.5': tensor(0.9431),
 'test_precision_at_0.7': tensor(0.9483),
 'test_precision_at_0.9': tensor(0.9515),
 'test_recall_at_0.3': tensor(0.9495),
 'test_recall_at_0.5': tensor(0.9449),
 'test_recall_at_0.7': tensor(0.9421),
 'test_recall_at_0.9': tensor(0.9355)}